In [3]:
from numpy.core.einsumfunc import _einsum_path_dispatcher
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier 
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier

In [5]:
trainDataAddress = r"C:\Users\FIRAT.KURT\Documents\Thesis_Data\TrainDatas\FeatureSelection_20.csv"
df = pd.read_csv(trainDataAddress)


In [16]:
#df.head()
df.iloc[:,1:-2].head()

,AC005077.4,AC100771.2,AC124067.4,ADAMTS5,AGR3,AR,CDK1,CHST3,CXXC5,ERGIC1,FAM171A1,FOXM1,FSIP1,KCNJ11,KCNK15,LRRN4CL,PAMR1,RGMA,SMIM14,STOX2
0,2.129507,0.007742,0.728267,1.510309,138.145113,17.574259,16.130439,3.777593,29.546005,18.432280,2.304468,10.360540,6.193542,7.136308,12.919290,0.840148,0.410596,0.480948,39.315238,0.051246
1,0.850671,0.091134,0.386388,0.532894,120.993979,11.812889,10.599441,2.145143,38.852568,31.293555,1.114885,13.105041,1.250841,6.262420,4.940484,0.250029,1.390545,0.162853,64.035186,0.056391
2,1.777356,0.032846,0.602009,1.680550,68.024036,18.187149,7.870393,3.676316,43.460446,21.284913,2.809691,5.350194,19.716850,7.561253,26.344173,1.668778,1.018426,0.732698,43.398505,0.204818
3,0.538242,0.016022,1.098536,1.285384,193.557406,12.108335,5.109818,2.171825,69.186238,17.280963,4.149877,3.924920,19.928225,3.172062,11.133883,0.708183,0.898765,0.300714,31.800631,0.542957
4,2.648847,0.064049,0.587958,1.163992,79.260960,28.416958,6.162760,3.972258,44.761534,51.976293,3.117372,7.008434,4.353891,10.720579,29.206439,1.400175,1.926167,2.213332,80.667734,0.567483


In [17]:
def run(trial):
    fold = 0
    learning_rate = trial.suggest_float("learning_rate", 1e-2, 0.25, log=True)
    reg_lambda = trial.suggest_loguniform("reg_lambda", 1e-8, 100.0)
    reg_alpha = trial.suggest_loguniform("reg_alpha", 1e-8, 100.0)
    subsample = trial.suggest_float("subsample", 0.1, 1.0)
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.1, 1.0)
    max_depth = trial.suggest_int("max_depth", 1, 7)

    xtrain = df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)

    y_train = xtrain.Subtype
    y_valid = xvalid.Subtype

    xtrain = xtrain.iloc[:, 1:-2]
    xvalid = xvalid.iloc[:,1:-2]

    le = LabelEncoder()
    ytrain = pd.DataFrame(le.fit_transform(y_train))
    yvalid = pd.DataFrame(le.transform(y_valid))
    minMaxScaler = MinMaxScaler()
    X_train = pd.DataFrame(minMaxScaler.fit_transform(xtrain), index=xtrain.index,columns=xtrain.columns)
    model = XGBClassifier(
        random_state=42,
        n_estimators=7000,
        objective="multi:softmax",
        learning_rate=learning_rate,
        reg_lambda=reg_lambda,
        reg_alpha=reg_alpha,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        max_depth=max_depth,
    )
    model.fit(xtrain, ytrain, early_stopping_rounds=300, eval_set=[(xvalid, yvalid)], verbose=1000)
    preds_valid = model.predict(xvalid)
    prec =  precision_score(yvalid,preds_valid, average='macro')
    return prec

In [18]:
import optuna
study = optuna.create_study(direction='maximize')
study.optimize(run, n_trials=50)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)


[I 2021-10-31 22:59:15,695] A new study created in memory with name: no-name-d79bb572-55b9-4827-bf81-567d81d5b196


[0]	validation_0-merror:0.85000
Will train until validation_0-merror hasn't improved in 300 rounds.


C:\Users\FIRAT.KURT\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Stopping. Best iteration:
[0]	validation_0-merror:0.85000



C:\Users\FIRAT.KURT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2021-10-31 22:59:16,040] Trial 0 finished with value: 0.024999999999999998 and parameters: {'learning_rate': 0.03172943914035018, 'reg_lambda': 2.40622912672306e-05, 'reg_alpha': 92.11986347760013, 'subsample': 0.2024408276134842, 'colsample_bytree': 0.5179963581153856, 'max_depth': 1}. Best is trial 0 with value: 0.024999999999999998.


...

[0]	validation_0-merror:0.51111
Will train until validation_0-merror hasn't improved in 300 rounds.


C:\Users\FIRAT.KURT\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


Stopping. Best iteration:
[10]	validation_0-merror:0.45000



C:\Users\FIRAT.KURT\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
[I 2021-10-31 23:00:32,168] Trial 49 finished with value: 0.17379336832671874 and parameters: {'learning_rate': 0.01463222302141779, 'reg_lambda': 0.001672242860103889, 'reg_alpha': 90.69589324110983, 'subsample': 0.636283037052918, 'colsample_bytree': 0.8214782615344693, 'max_depth': 5}. Best is trial 10 with value: 0.8587205387205388.


Number of finished trials: 50
Best trial: {'learning_rate': 0.04586090618716276, 'reg_lambda': 0.06826522569951803, 'reg_alpha': 1.7871177682650604e-06, 'subsample': 0.40807207936359097, 'colsample_bytree': 0.3135487605486668, 'max_depth': 7}
